# PyTorch Lightning Faster R-CNN Inference LB=0.358
Run inference with trained model from [Starter PyTorch Lightning Faster R-CNN Finetuning](https://www.kaggle.com/clemchris/starter-pytorch-lightning-faster-r-cnn-finetuning).

Inference notebooks cannot use the internet. Therefore, we need to upload the required wheels that are not included in Kaggle's Docker image, pretrained weights, and model checkpoints ourselves using Kaggle Datasets.

## Sources and Inspirations
- [Reef- Starter Torch FasterRCNN Infer [LB=0.416]](https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-416)

## Imports

In [ ]:
import math
from pathlib import Path

import greatbarrierreef
import numpy as np
import pytorch_lightning as pl
import torch
import torchmetrics
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn

## Paths

In [ ]:
INPUT_DIR = Path("../input")

CHECKPOINTS_DIR = INPUT_DIR / "fasterrcnn-checkpoints"

PRETRAINED_WEIGHTS_PATH = CHECKPOINTS_DIR / "fasterrcnn_resnet50_fpn_coco-258fb6c6.pth"
FINETUNED_CHECKPOINT_PATH = CHECKPOINTS_DIR / "model.ckpt"


## Settings

In [ ]:
DETECTION_THRESHOLD = 0.66

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Lightning Module Class

In [ ]:
class GBRModule(pl.LightningModule):
    """LightningModule class to finetune torchvision's Faster R-CNN model."""
    
    def __init__(self, pretrained_weights_path=None):
        super().__init__()

        self.model = self._create_model(pretrained_weights_path)

        # self.val_map = torchmetrics.MAP()

    def _create_model(self, pretrained_weights_path):
        """Creates finetunable Faster R-CNN model."""
        if pretrained_weights_path is None:
            model = fasterrcnn_resnet50_fpn(pretrained=True)
        else:
            model = fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
            model.load_state_dict(torch.load(pretrained_weights_path))

        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

        return model

    def forward(self, image):
        """Runs inference."""
        self.model.eval()
        output = self.model(image)

        return output

    def training_step(self, batch, batch_idx):
        image, target = batch
        loss_dict = self.model(image, target)
        losses = sum(loss for loss in loss_dict.values())

        batch_size = len(batch[0])
        self.log_dict(loss_dict, batch_size=batch_size)
        self.log("train_loss", losses, batch_size=batch_size)

        return losses

    def validation_step(self, batch, batch_idx):
        image, target = batch
        output = self.model(image)

        #val_map = self.val_map(output, target)
        #self.log("val_map", val_map["map"])

    def configure_optimizers(self):
        params = [p for p in self.model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

        return [optimizer], [lr_scheduler]

## Helper Functions

In [ ]:
def get_eval_gbr_module(checkpoint_path, pretrained_weights_path, device):
    gbr_module = GBRModule.load_from_checkpoint(checkpoint_path, pretrained_weights_path=pretrained_weights_path)
    gbr_module.eval()
    
    gbr_module = gbr_module.to(device)
    
    return gbr_module

In [ ]:
def create_prediction_string(boxes, scores):
    pred_strings = []
    for box, score in zip(boxes, scores):
        x, y, w, h = box
        pred_string = f"{score:.2f} {x} {y} {w} {h}"
        pred_strings.append(pred_string)
    
    pred_string = " ".join(pred_strings)
    return pred_string

In [ ]:
# From: https://www.kaggle.com/julian3833/reef-starter-torch-fasterrcnn-infer-lb-0-416
def predict(model, pixel_array, detection_threshold):
    # Convert to tensor
    pixel_array = pixel_array.astype(np.float32) / 255.
    image = torch.from_numpy(pixel_array.transpose(2, 0, 1)).unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(image.to(model.device))[0]
    
    # Move predictions to cpu and numpy
    boxes = outputs['boxes'].data.cpu().numpy()
    scores = outputs['scores'].data.cpu().numpy()
    
    # Filter predictions with low score
    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    scores = scores[scores >= detection_threshold]
    
    # Go back from x_min, y_min, x_max, y_max to x_min, y_min, w, h
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
  
    prediction_string = create_prediction_string(boxes, scores)
    return prediction_string

## Create Submission env

In [ ]:
env = greatbarrierreef.make_env()
iter_test = env.iter_test() 

## Predict

In [ ]:
eval_gbr_module = get_eval_gbr_module(FINETUNED_CHECKPOINT_PATH, PRETRAINED_WEIGHTS_PATH, DEVICE)

In [ ]:
for (pixel_array, df_pred) in iter_test:  
    prediction_string = predict(eval_gbr_module, pixel_array, DETECTION_THRESHOLD)
    print(prediction_string)
    df_pred['annotations'] = prediction_string
    env.predict(df_pred)